In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install "unsloth[colab_ampere] @ git+https://github.com/unslothai/unsloth.git"
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"
pass

!pip install "git+https://github.com/huggingface/transformers.git" # Native 4bit loading works!

In [ ]:
# !pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 tensorboard huggingface_hub[cli] xformers

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import os
import torch
from datasets import load_dataset

from unsloth import FastLanguageModel
from peft import LoraConfig
from trl import SFTTrainer
import transformers

## Restart your session

In [ ]:
dtype = None
max_seq_length=2048
# Load the base model
model,tokenizer = FastLanguageModel.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    device_map="auto",
    max_seq_length = max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)


Unsloth: You passed in `mistralai/Mistral-7B-v0.1` and `load_in_4bit = True`.
We shall load `unsloth/mistral-7b-bnb-4bit` for 4x faster loading.


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# SECTION 1.2
## Setting up Dataset
- The Llama-chat model expects prompts of the form:
[INST] {user_message} [/INST]

To train the model, we construct our training data like this:
[INST] {user_message} [/INST] {response}

In [ ]:
from datasets import load_dataset

dataset_name = 'Amod/mental_health_counseling_conversations'
dataset = load_dataset(dataset_name, split="train")

In [ ]:
def formatting_prompts_func(batch):
    texts = [value.replace("<start>", "").replace("<end>", "").strip() for value in batch["Context"]]
    return {"Context": texts}

In [ ]:
dataset = dataset.map(formatting_prompts_func,batched=True)

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['Context', 'Response'],
    num_rows: 3512
})

In [ ]:
from IPython.display import Markdown
Markdown(dataset["Response"][0])

If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media.  Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is somehow terrible.Bad feelings are part of living.  They are the motivation to remove ourselves from situations and relationships which do us more harm than good.Bad feelings do feel terrible.   Your feeling of worthlessness may be good in the sense of motivating you to find out that you are much better than your feelings today.

In [ ]:
Markdown(dataset["Context"][0])


I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?

In [ ]:
alpaca_prompt = """Below is an input that describes a task and provides further context. Write a response that appropriately completes the request.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["Context"]
    responses      = examples["Response"]
    texts = []
    for instruction, input in zip(instructions, responses):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
# dataset = dataset.map(formatting_prompts_func, batched = True,)


dataset = dataset.map(formatting_prompts_func, remove_columns=[f for f in dataset.features if not f == 'text'],batched=True)

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [ ]:
Markdown(dataset["text"][0])

Below is an input that describes a task and provides further context. Write a response that appropriately completes the request.

### Input:
I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?

### Response:
If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media.  Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is somehow terrible.Bad feelings are part of living.  They are the motivation to remove ourselves from situations and relationships which do us more harm than good.Bad feelings do feel terrible.   Your feeling of worthlessness may be good in the sense of motivating you to find out that you are much better than your feelings today.</s>

In [ ]:
new_model = 'mistral-7b-medical-assistance'

# SECTION 1.3 Training with LORA

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 4,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=4):   0%|          | 0/3512 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,512 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 2
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.877000
2,1.905000
3,2.016700
4,2.021400
5,1.916900
6,1.780700
7,1.855400
8,1.960500
9,2.062800
10,1.894800


In [ ]:
# FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Suggest remedies for me. I am currently suffering from stress due to studies. What should I do?.", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 200 ) # optionally set temperature and do_sample=True
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


["<s> Below is an input that describes a task and provides further context. Write a response that appropriately completes the request.\n\n### Input:\nSuggest remedies for me. I am currently suffering from stress due to studies. What should I do?.\n\n### Response:\nI'm sorry to hear that you are feeling stressed out. \xa0I'm sure you are not alone in feeling this way. \xa0I would suggest that you take a break from your studies and do something that you enjoy. \xa0This could be anything from going for a walk, to watching a movie, to reading a book. \xa0I would also suggest that you talk to someone about how you are feeling. \xa0This could be a friend, family member, or a counselor. \xa0Talking to someone can help you to feel better and can also help you to come up with a plan to deal with your stress. \xa0I hope that you are able to find some relief from your stress.</s>"]

In [ ]:
model.save_pretrained_merged(new_model, tokenizer, save_method = "merged_16bit")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.35 out of 12.67 RAM for saving.


100%|██████████| 32/32 [00:34<00:00,  1.06s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving mistral-7b-medical-assistance/pytorch_model-00001-of-00003.bin...
Unsloth: Saving mistral-7b-medical-assistance/pytorch_model-00002-of-00003.bin...
Unsloth: Saving mistral-7b-medical-assistance/pytorch_model-00003-of-00003.bin...
Done.


In [ ]:
model.push_to_hub_merged(new_model, tokenizer, save_method = "merged_16bit")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.97 out of 12.67 RAM for saving.


100%|██████████| 32/32 [00:56<00:00,  1.78s/it]


Unsloth: Saving tokenizer...

RepositoryNotFoundError: 404 Client Error. (Request ID: Root=1-660ea5e4-221046ab5381c1ca277ab00b;1981e126-e15d-425a-be11-93d788a31589)

Repository Not Found for url: https://huggingface.co/api/models/mistral-7b-medical-assistance.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.